In [134]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen, urlretrieve, quote
from urllib.parse import urljoin
import os
import wget
from glob import *

# 1. Use WebScraping to download dataset

In [130]:
OUTPUT_DIR = 'C:/Users/theri/OneDrive/datascience_projects'
URL = 'https://www.eia.gov/electricity/data/eia861m/'
u = urlopen(URL)

try:
    html = u.read().decode('utf-8')
finally:
    u.close()

#parse text obtained
soup = bs(html, "html.parser")

In [131]:
#requests URL and get response object
#response = requests.get(URL)

In [132]:
#find all sales and revenue hyperlinks on page
links =  soup.select('a[title^="Sales and revenue"]')

In [133]:
#from all links download file
for a in links:
    print("Found the URL:", a['href'])
    _FULLURL = URL + a['href']
    href = a.get('href')
    href = URL + href
    wget.download(href)
    #filename = os.path.join(OUTPUT_DIR,href.rsplit('/',1)[-1])
    #print("Downloading %s to %s..." % (href, filename))
    #urlretrieve(href,filename)
    #print("Done.")
    
    


Found the URL: xls/sales_revenue.xlsx
100% [..........................................................................] 4575362 / 4575362Found the URL: xls/sales_ult_cust_2022.xlsx
100% [............................................................................] 595240 / 595240Found the URL: archive/xls/retail_sales_2021.xlsx
100% [..........................................................................] 1409300 / 1409300Found the URL: archive/xls/retail_sales_2020.xlsx
100% [..........................................................................] 1436351 / 1436351Found the URL: archive/xls/retail_sales_2019.xlsx
100% [..........................................................................] 1442566 / 1442566Found the URL: archive/xls/retail_sales_2018.xlsx
100% [..........................................................................] 1437953 / 1437953Found the URL: archive/xls/retail_sales_2017.xlsx
100% [....................................................................

# 2. Data Cleanse

In [143]:
#assign all data file names to a list
file_names = glob('*[.xls,.xlsx]')

In [145]:
#remove the data file that contains aggregated data from the file list
new_file_names = []
for file in file_names:
    if 'revenue' in file:
        print ('dont need this file')
    else:
        new_file_names.append(file)

dont need this file


In [147]:
# get subset of data that contains data from year 2003 to 2022
new_file_names = new_file_names[13:]

In [158]:
#create empty dataframe
df_data = pd.DataFrame()

#append data files to empty dataframe
for file in new_file_names:
    data = pd.read_excel(file)
    print(file)
    print(data.columns)
    print(data.shape)
    df_data = df_data.append(data)

f8262003.xls
Index(['UTILITYID', 'UTILNAME', 'STATE', 'YEAR', 'MONTH',
       'RES_REV (Thousand $)', 'RES_SALES (MwH)', 'RES_CONS ',
       'COM_REV (Thousand $)', 'COM_SALES (MwH)', 'COM_CONS',
       'IND_REV (Thousand $)', 'IND_SALES (MwH)', 'IND_CONS',
       'OTH_REV (Thousand $)', 'OTH_SALES (MwH)', 'OTH_CONS',
       'TOT_REV (Thousand $)', 'TOT_SALES (MwH)', 'TOT_CONS'],
      dtype='object')
(6412, 20)
f8262004.xls
Index(['UTILITYID', 'UTILNAME', 'STATE', 'YEAR', 'MONTH',
       'RES_REV (Thousand $)', 'RES_SALES (MwH)', 'RES_CONS ',
       'COM_REV (Thousand $)', 'COM_SALES (MwH)', 'COM_CONS',
       'IND_REV (Thousand $)', 'IND_SALES (MwH)', 'IND_CONS',
       'OTH_REV (Thousand $)', 'OTH_SALES (MwH)', 'OTH_CONS',
       'TOT_REV (Thousand $)', 'TOT_SALES (MwH)', 'TOT_CONS'],
      dtype='object')
(6499, 20)
f8262005.xls
Index(['UTILITYID', 'UTILNAME', 'STATE', 'YEAR', 'MONTH',
       'RES_REV (Thousand $)', 'RES_SALES (MwH)', 'RES_CONS ',
       'COM_REV (Thousand $)', 'CO

In [163]:
df_2013 = pd.read_excel('f8262013.xls',header=2)

In [164]:
df_2013.head()

,Year,Month,Utility Number,Utility Name,State,Data Status,Thousands Dollars,Megawatthours,Count,Thousands Dollars.1,...,Count.1,Thousands Dollars.2,Megawatthours.2,Count.2,Thousands Dollars.3,Megawatthours.3,Count.3,Thousands Dollars.4,Megawatthours.4,Count.4
0,2013,1.0,0.0,Adjustment 2014,AK,Final,6598.015,25303.99,30971,9177.412,...,11768,2065.652,9197.801,548,0,0,0,17841.079,68189.131,43287.0
1,2013,1.0,213.0,Alaska Electric Light&Power Co,AK,Final,2290.654,16227.499,13901,1458.255,...,2186,594.351,5270.856,94,0,0,0,4343.260,33661.999,16181.0
2,2013,1.0,219.0,Alaska Power and Telephone Co,AK,Final,738.549,2449.644,4745,891.519,...,2126,0,0,0,0,0,0,1630.068,5683.881,6871.0
3,2013,1.0,599.0,Anchorage Municipal Light and Power,AK,Final,1632.122,14825.838,24423,6906.278,...,6364,0,0,0,0,0,0,8538.400,98206.785,30787.0
4,2013,1.0,1651.0,Bethel Utilities Corp,AK,Final,574.107,1052.715,1692,1413.971,...,975,0,0,0,0,0,0,1988.078,3743.974,2667.0
